In [1]:
import numpy as np
from ondelette import wavelets
import pywt
import tensorflow as tf

np.set_printoptions(floatmode='unique')
rng = np.random.default_rng()

2025-02-21 19:48:30.557486: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
dtype = tf.float64
wavelet_name = 'db3'

wavelet = wavelets.by_name(wavelet_name, dtype=dtype)
wavelet_width = wavelet.shape[0]

samples_np = rng.integers(-32768, 32767, [2, 36], dtype=np.int16, endpoint=True)
samples = tf.constant(samples_np, dtype=dtype)
print(f'SAMPLES: {samples.shape}')
print(samples.numpy())
print()

a3, d3, d2, d1, d0 = wavelets.wavedec(wavelet, 5, samples)

r2 = wavelets.idwt(wavelet, a3, d3)
print(f'r2: {r2.shape}')
print(r2.numpy())
print()
print(f'd2: {d2.shape}')
print(d2.numpy())
print()
pre_pad_a = 4
pre_pad_d = 2
r2 = tf.pad(r2, [[0, 0], [pre_pad_a, pre_pad_a]])
d2 = tf.pad(d2, [[0, 0], [pre_pad_d, pre_pad_d]])
r1 = wavelets.idwt(wavelet, r2, d2)
print(f'r1: {r1.shape}')
print(r1.numpy())
print()
print(f'd1: {d1.shape}')
print(d1.numpy())
print()
pre_pad_a = 2
pre_pad_d = 4
r1 = tf.pad(r1, [[0, 0], [pre_pad_a, pre_pad_a]])
d1 = tf.pad(d1, [[0, 0], [pre_pad_d, pre_pad_d]])
r0 = wavelets.idwt(wavelet, r1, d1)
print(f'r0: {r0.shape}')
print(r0.numpy())
print()
print(f'd0: {d0.shape}')
print(d0.numpy())
print()
pre_pad_a = 2
pre_pad_d = 8
r0 = tf.pad(r0, [[0, 0], [pre_pad_a, pre_pad_a]])
d0 = tf.pad(d0, [[0, 0], [pre_pad_d, pre_pad_d]])
rec_float = wavelets.idwt(wavelet, r0, d0)
rec = tf.round(rec_float)
print(f'RECONSTRUCTION: {rec.shape}')
print(rec.numpy())
print()


SAMPLES: (2, 36)
[[-19276. -15212. -13306.  12886.  28973.   2766.  29238.  30622. -18561.
   -1670.   -627. -30156.   6719. -14269.   3969.  31775. -20376. -19863.
  -14323.  -1527.  13013.  23147. -11170.  24594.  24230.   2218. -10008.
  -21735. -11042.  -2088.  23290. -26957.   4065.   5826. -15375. -16546.]
 [ 11799.  28838.   7886.   5618.  12810.   1516. -23054. -17593. -22096.
    9487.  13289.  -7228.   3290. -30059. -19481.  23254. -31074. -26759.
     680. -26310. -13135. -10863. -31512.   9357.  31790. -26688. -12688.
  -24138.  -5509. -18086. -19785.  -4432.  24292.  22052.  20341.  24264.]]

r2: (2, 8)
[[   175.4210233191651    1445.81415234133    -10926.693434213139
   18576.057642008018    -9699.89165648146      8547.502096511165
   -1069.189785148251   -12973.16065111772   ]
 [   -77.76059966674322    243.97339533146942   1765.6092751019403
   14107.081766195644   -16234.51496503229    -34859.44664897858
  -33114.02479755376     25768.48510092276   ]]

d2: (2, 12)
[[  

In [2]:
dtype = tf.float32
wavelet_name = 'db3'

wavelet = wavelets.by_name(wavelet_name, dtype=dtype)
wavelet_width = wavelet.shape[0]

def random_pad(tensor, width):
    min = tf.reduce_min(tensor)
    max = tf.reduce_max(tensor)
    scale = max = min
    left_np = rng.random([2, width]) * scale - min
    left = tf.constant(left_np, dtype=tensor.dtype)
    right_np = rng.random([2, width]) * scale - min
    right = tf.constant(right_np, dtype=tensor.dtype)
    return tf.concat([left, tensor, right], axis=1)


samples_np = rng.integers(-32768, 32767, [2, 20], dtype=np.int16, endpoint=True)
samples_unpadded = tf.constant(samples_np, dtype=dtype)
samples = samples_unpadded #tf.pad(samples_unpadded, [[0, 0], [16, 16]])
print(f'SAMPLES: {samples.shape}')
print(samples.numpy())
print()

dec0 = wavelets.dwt(wavelet, samples)
a0, d0 = dec0
dec1 = wavelets.dwt(wavelet, a0)
a1, d1 = dec1
dec2 = wavelets.dwt(wavelet, a1)
a2, d2 = dec2

d1_padding = ((d2.shape[1] * 2) - d1.shape[1]) // 2
d1 = random_pad(d1, d1_padding) #tf.pad(d1, [[0, 0], [d1_padding, d1_padding]])

d0_padding = ((d1.shape[1] * 2) - d0.shape[1]) // 2
d0 = random_pad(d0, d0_padding) # tf.pad(d0, [[0, 0], [d0_padding, d0_padding]])

# print(f'd2: {d2.shape}')
# # print(d2.numpy())
# # print()
# print(f'a2: {a2.shape}')
# # print(a2.numpy())
# # print()


print(f'a2: {a2.shape}')
print(a2.numpy())
print()
print(f'd2: {d2.shape}')
print(d2.numpy())
print()
r1 = wavelets.idwt(wavelet, a2, d2)
print(f'a1: {a1.shape}')
print(a1.numpy())
print()
print(f'r1: {r1.shape}')
print(r1.numpy())
print()
print(f'd1: {d1.shape}')
print(d1.numpy())
print()
pre_pad_a = 4
pre_pad_d = 2
r1 = tf.pad(r1, [[0, 0], [pre_pad_a, pre_pad_a]])
d1 = tf.pad(d1, [[0, 0], [pre_pad_d, pre_pad_d]])
r0 = wavelets.idwt(wavelet, r1, d1)
print(f'a0: {a0.shape}')
print(a0.numpy())
print()
print(f'r0: {r0.shape}')
print(r0.numpy())
print()
print(f'd0: {d0.shape}')
print(d0.numpy())
print()
pre_pad_a = 2
pre_pad_d = 4
r0 = tf.pad(r0, [[0, 0], [pre_pad_a, pre_pad_a]])
d0 = tf.pad(d0, [[0, 0], [pre_pad_d, pre_pad_d]])
rec_float = wavelets.idwt(wavelet, r0, d0)
rec = tf.round(rec_float)
print(f'RECONSTRUCTION: {rec.shape}')
print(rec.numpy())
print()

SAMPLES: (2, 20)
[[-20041. -15260.  16486.   2046. -21201. -10963. -21719.  -8540.  -5053.
   21308. -18069.  31962.  -9977. -25562.   9969. -15965.  -7869.   7370.
  -10645.  23237.]
 [  8805.  27474. -22560. -29207.  26465.  15527.  -8348.  -3060.  15858.
   16682.  -2510.  -7899. -20421.  12835.  15748.   3008.  17165.  -3241.
    9282.  13998.]]

a2: (2, 6)
[[   124.92962    -91.54471   1135.812   -21472.623    -9538.903
    2093.3396 ]
 [   -80.88053   -742.6271    1856.9187    7118.7134    5897.8423
   16214.554  ]]

d2: (2, 6)
[[ -1179.812    -5288.2573  -13813.806   -22458.262    -2166.052
     221.66254]
 [   763.8206   10996.602    17328.723     9284.964    -8329.475
    1716.9498 ]]

a1: (2, 8)
[[  -399.53882    2577.4075    -9806.309    -11289.605    -21335.566
    9980.617    -19676.818     10706.814   ]
 [    25.220001  -2234.8562    10146.163     -5366.851     17512.527
   -6883.156     16175.169     13426.28    ]]

r1: (2, 8)
[[  -399.53876    2577.4072    -9806.308    

In [38]:
dtype = tf.float32
wavelet_name = 'db3'

wavelet = wavelets.by_name(wavelet_name, dtype=dtype)
wavelet_width = wavelet.shape[0]

samples_np = rng.integers(-32768, 32767, [2, 36], dtype=np.int16, endpoint=True)
samples_unpadded = tf.constant(samples_np, dtype=dtype)
samples = samples_unpadded #tf.pad(samples_unpadded, [[0, 0], [16, 16]])
print(f'SAMPLES: {samples.shape}')
print(samples.numpy())
print()

dec0 = wavelets.dwt(wavelet, samples)
a0, d0 = dec0
dec1 = wavelets.dwt(wavelet, a0)
a1, d1 = dec1
dec2 = wavelets.dwt(wavelet, a1)
a2, d2 = dec2
dec3 = wavelets.dwt(wavelet, a2)
a3, d3 = dec3

d2_padding = ((d3.shape[1] * 2) - d2.shape[1]) // 2
d2 = random_pad(d2, d2_padding)

d1_padding = ((d2.shape[1] * 2) - d1.shape[1]) // 2
d1 = random_pad(d1, d1_padding) #tf.pad(d1, [[0, 0], [d1_padding, d1_padding]])

d0_padding = ((d1.shape[1] * 2) - d0.shape[1]) // 2
d0 = random_pad(d0, d0_padding) # tf.pad(d0, [[0, 0], [d0_padding, d0_padding]])

# print(f'a3: {a3.shape}')
# print(f'd3: {d3.shape}')
r2 = wavelets.idwt(wavelet, a3, d3)
print(f'a2: {a2.shape}')
print(a2.numpy())
print()
print(f'r2: {r2.shape}')
print(r2.numpy())
print()
print(f'd2: {d2.shape}')
print(d2.numpy())
print()
pre_pad_a = 4
pre_pad_d = 2
r2 = tf.pad(r2, [[0, 0], [pre_pad_a, pre_pad_a]])
d2 = tf.pad(d2, [[0, 0], [pre_pad_d, pre_pad_d]])
r1 = wavelets.idwt(wavelet, r2, d2)
print(f'a1: {a1.shape}')
print(a1.numpy())
print()
print(f'r1: {r1.shape}')
print(r1.numpy())
print()
print(f'd1: {d1.shape}')
print(d1.numpy())
print()
pre_pad_a = 2
pre_pad_d = 4
r1 = tf.pad(r1, [[0, 0], [pre_pad_a, pre_pad_a]])
d1 = tf.pad(d1, [[0, 0], [pre_pad_d, pre_pad_d]])
r0 = wavelets.idwt(wavelet, r1, d1)
print(f'a0: {a0.shape}')
print(a0.numpy())
print()
print(f'r0: {r0.shape}')
print(r0.numpy())
print()
print(f'd0: {d0.shape}')
print(d0.numpy())
print()
pre_pad_a = 2
pre_pad_d = 8
r0 = tf.pad(r0, [[0, 0], [pre_pad_a, pre_pad_a]])
d0 = tf.pad(d0, [[0, 0], [pre_pad_d, pre_pad_d]])
rec_float = wavelets.idwt(wavelet, r0, d0)
rec = tf.round(rec_float)
print(f'RECONSTRUCTION: {rec.shape}')
print(rec.numpy())
print()

SAMPLES: (2, 36)
[[ 2.3726e+04  1.6872e+04  2.3529e+04 -1.3051e+04 -2.3487e+04 -2.9650e+04
   2.0930e+03 -3.0830e+03  1.4798e+04 -6.6920e+03 -7.8570e+03  3.2860e+03
   1.7687e+04 -1.8419e+04  1.3390e+03  1.3634e+04 -5.3370e+03  1.8742e+04
   2.3444e+04 -5.4240e+03  1.0135e+04 -2.8236e+04  7.6800e+03  3.1833e+04
  -3.1280e+04 -1.7401e+04  3.1966e+04 -2.8000e+01 -3.2205e+04 -2.5994e+04
   1.9202e+04  1.2822e+04  2.9485e+04  2.5479e+04  2.8674e+04 -2.1063e+04]
 [ 2.8949e+04  1.2185e+04  2.3979e+04  2.9697e+04 -1.7433e+04 -2.3447e+04
   1.1619e+04  7.4330e+03  1.3847e+04 -2.5998e+04 -2.4676e+04 -2.3553e+04
   3.0255e+04 -2.9979e+04  1.6877e+04 -5.0300e+03  3.2330e+03 -2.1100e+03
   3.1671e+04  2.2836e+04  1.7568e+04 -1.6494e+04 -2.3091e+04  1.5219e+04
  -3.9060e+03  1.8587e+04  1.4960e+04 -2.7690e+04  2.3910e+03  2.9494e+04
   7.3830e+03 -3.1783e+04 -4.0580e+03  2.3448e+04 -9.2250e+03 -2.3516e+04]]

a2: (2, 8)
[[  -199.17682  -1700.4159    9894.589    -5257.0186     282.02707
   13109.7   

On the way down, after we decompose the approximation:
* pad the detail, on center, to be twice the width of the approximation.